In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
with open('../data/training_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('../data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

with open('../data/training_labels.pkl', 'rb') as f:
    train_labels = pickle.load(f)

In [3]:
class knn:
        
        def __init__(self, kernel, k=3) -> None:
            self.k = k
            self.kernel = kernel
            
        def fit(self, X, y):
            self.X = X
            self.y = y

            return self
        
        def compute_distance(self, x, base):
            k = self.kernel(x,x)
            return [k+self.kernel(b,b)-2*self.kernel(x, b) for b in base]
        
        def predict(self, X):
            y_pred = []
            score_list = []
            for x in tqdm(X):
                scores = np.array(self.compute_distance(x, self.X))
                idx = np.argsort(scores)[:self.k]
                y_pred.append((self.y[idx].mean()>.5).astype(int))
                score_list.append(scores)
            return np.array(y_pred), np.array(score_list)
        
        def score(self, X, y):
            return np.mean(self.predict(X)[0]==y)

In [4]:
import sys 
import os

if 'src' not in [el.split('/')[-1] for el in sys.path]:
    curr_path = os.getcwd()
    sys.path.append('/'.join((curr_path.split('/')[:-1])))

In [8]:
%pip install scikit-learn

  Using cached scikit_learn-1.2.2-cp311-cp311-macosx_10_9_x86_64.whl (9.0 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [9]:
from src.kernels import RandomWalkKernel, QuadraticEdgeKernel, VertexHistogramKernel
from sklearn.model_selection import train_test_split


In [12]:
idx1 = train_labels==1
idx0 = train_labels==0

train_data = np.array(train_data, dtype=object)

X_train = train_data[idx1][:30]
X_train = np.concatenate((X_train, train_data[idx0][:30]))
y_train = np.concatenate((np.ones(30), np.zeros(30)))

X_test = train_data[idx1][30:60]
X_test = np.concatenate((X_test, train_data[idx0][30:60]))
y_test = np.concatenate((np.ones(30), np.zeros(30)))


X_train, _, y_train, _ = train_test_split(train_data, train_labels, test_size=0.5)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

#knn_ = knn(k=3, kernel=RandomWalkKernel(edge_kernel=QuadraticEdgeKernel())).fit(X_train, y_train)
knn_ = knn(k=3, kernel=VertexHistogramKernel()).fit(X_train, y_train)


In [13]:
a = knn_.predict(X_test)
np.mean(a[0]==y_test)

100%|██████████| 600/600 [05:09<00:00,  1.94it/s]


0.8833333333333333

In [15]:
#print class distribution
print('Train class distribution: ', np.bincount(y_train.astype(int)))
print('Test class distribution: ', np.bincount(y_test.astype(int)))

Train class distribution:  [2187  213]
Test class distribution:  [539  61]


In [23]:
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, a[0])

0.5279965935703641